In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/genai-toolbox/blob/main/docs/en/getting-started/colab_quickstart.ipynb)

# Getting Started With MCP Toolbox

This guide demonstrates how to quickly run
[Toolbox](https://github.com/googleapis/genai-toolbox) end-to-end in Google
Colab using Python, PostgreSQL, and either [Google
GenAI](https://pypi.org/project/google-genai/), [ADK](https://google.github.io/adk-docs/),
[Langgraph](https://www.langchain.com/langgraph)
or [LlamaIndex](https://www.llamaindex.ai/).

Within this Colab environment, you'll
- Set up a `PostgreSQL database`.
- Launch a Toolbox server.
- Connect to Toolbox and develop a sample `Hotel Booking` application.

Here is the simplified flow of a Toolbox Application:

<img src="https://services.google.com/fh/files/misc/toolbox_flow.png" alt="Toolbox Flow"/>



## Step 1: Set up your database

In this section, we will
1. Create a database.
1. Create a user to access the database.
1. Insert example data into the database.

In [ ]:
# Install postgresql to run a DB server on colab
%%shell

sudo apt-get -y -qq update > /dev/null 2>&1
sudo apt-get -y -qq install postgresql > /dev/null 2>&1

In [ ]:
# Start the postgresql server.
!sudo service postgresql start

In [ ]:
# Check that postgres is running
!sudo lsof -i :5432

In [ ]:
# Create a dedicated database and a user to access our DB securely
%%shell

sudo -u postgres psql << EOF
CREATE USER toolbox_user WITH PASSWORD 'my-password';
CREATE DATABASE toolbox_db;
GRANT ALL PRIVILEGES ON DATABASE toolbox_db TO toolbox_user;
ALTER DATABASE toolbox_db OWNER TO toolbox_user;
EOF

> **Tip:** For a real application, it’s best to follow the principle of least permission and only grant the privileges your application needs.



In [ ]:
# Connect to the database with the new user and create a hotels table.
%%shell

export PGPASSWORD=my-password
psql -h 127.0.0.1 -U toolbox_user -d toolbox_db --no-password << EOF
CREATE TABLE hotels(
   id            INTEGER NOT NULL PRIMARY KEY,
   name          VARCHAR NOT NULL,
   location      VARCHAR NOT NULL,
   price_tier    VARCHAR NOT NULL,
   checkin_date  DATE    NOT NULL,
   checkout_date DATE    NOT NULL,
   booked        BIT     NOT NULL
);
INSERT INTO hotels(id, name, location, price_tier, checkin_date, checkout_date, booked)
VALUES
  (1, 'Hilton Basel', 'Basel', 'Luxury', '2024-04-22', '2024-04-20', B'0'),
  (2, 'Marriott Zurich', 'Zurich', 'Upscale', '2024-04-14', '2024-04-21', B'0'),
  (3, 'Hyatt Regency Basel', 'Basel', 'Upper Upscale', '2024-04-02', '2024-04-20', B'0'),
  (4, 'Radisson Blu Lucerne', 'Lucerne', 'Midscale', '2024-04-24', '2024-04-05', B'0'),
  (5, 'Best Western Bern', 'Bern', 'Upper Midscale', '2024-04-23', '2024-04-01', B'0'),
  (6, 'InterContinental Geneva', 'Geneva', 'Luxury', '2024-04-23', '2024-04-28', B'0'),
  (7, 'Sheraton Zurich', 'Zurich', 'Upper Upscale', '2024-04-27', '2024-04-02', B'0'),
  (8, 'Holiday Inn Basel', 'Basel', 'Upper Midscale', '2024-04-24', '2024-04-09', B'0'),
  (9, 'Courtyard Zurich', 'Zurich', 'Upscale', '2024-04-03', '2024-04-13', B'0'),
  (10, 'Comfort Inn Bern', 'Bern', 'Midscale', '2024-04-04', '2024-04-16', B'0');
SELECT * from hotels;
EOF

In [ ]:
# Check that database is running
!sudo lsof -i :5432

## Optional: Enable Vertex AI API for Google Cloud

If you're using a model hosted on **Vertex AI**, run the following command to enable the API:

```bash
!gcloud services enable aiplatform.googleapis.com


## Step 2: Install and configure Toolbox

In this section, we will
1. Download the latest version of the toolbox binary.
2. Create a toolbox config file.
3. Start a toolbox server using the config file.



Download the [latest](https://github.com/googleapis/genai-toolbox/releases) version of Toolbox as a binary.

In [ ]:
version = "0.27.0" # x-release-please-version
! curl -O https://storage.googleapis.com/genai-toolbox/v{version}/linux/amd64/toolbox

# Make the binary executable
! chmod +x toolbox

In [ ]:
TOOLBOX_BINARY_PATH = "/content/toolbox"
SERVER_PORT = 5000

> Note: To include a literal dollar sign (e.g., $1) as part of your SQL statement within the Python string for tools.yml, you must escape both the backslash and the dollar sign. Use \\\$1 in Python to output \$1 in the tools.yml file.

> Note: You can also set up Colab secrets to store any sensitive information like passwords. You can easily add secrets through the left panel:

<img src="https://services.google.com/fh/files/misc/colab_secret.png" alt="Colab Secrets" width="400"/>


Create a tools file with the following functions:

- `Database Connection (sources)`: `Includes details for connecting to our hotels database.`
- `Tool Definitions (tools)`: `Defines five tools for database interaction:`
  - `search-hotels-by-name`
  - `search-hotels-by-location`
  - `book-hotel`
  - `update-hotel`
  - `cancel-hotel`

Our application will leverage these tools to interact with the hotels database.

For detailed configuration options, please refer to the [Toolbox documentation](https://googleapis.github.io/genai-toolbox/getting-started/configure/).



In [ ]:
# Create a tools file at runtime.
# You can also upload a tools file and use that to run toolbox.
tools_file_name = "tools.yml"
file_content = f"""
kind: sources
name: my-pg-source
type: postgres
host: 127.0.0.1
port: 5432
database: toolbox_db
user: toolbox_user
password: my-password
---
kind: tools
name: search-hotels-by-name
type: postgres-sql
source: my-pg-source
description: Search for hotels based on name.
parameters:
  - name: name
    type: string
    description: The name of the hotel.
statement: SELECT * FROM hotels WHERE name ILIKE '%' || \$1 || '%';
---
kind: tools
name: search-hotels-by-location
type: postgres-sql
source: my-pg-source
description: Search for hotels based on location.
parameters:
  - name: location
    type: string
    description: The location of the hotel.
statement: SELECT * FROM hotels WHERE location ILIKE '%' || \$1 || '%';
---
kind: tools
name: book-hotel
type: postgres-sql
source: my-pg-source
description: >-
   Book a hotel by its ID. If the hotel is successfully booked, returns a NULL, raises an error if not.
parameters:
  - name: hotel_id
    type: string
    description: The ID of the hotel to book.
statement: UPDATE hotels SET booked = B'1' WHERE id = \$1;
---
kind: tools
name: update-hotel
type: postgres-sql
source: my-pg-source
description: >-
  Update a hotel's check-in and check-out dates by its ID. Returns a message
  indicating  whether the hotel was successfully updated or not.
parameters:
  - name: hotel_id
    type: string
    description: The ID of the hotel to update.
  - name: checkin_date
    type: string
    description: The new check-in date of the hotel.
  - name: checkout_date
    type: string
    description: The new check-out date of the hotel.
statement: >-
  UPDATE hotels SET checkin_date = CAST(\$2 as date), checkout_date = CAST(\$3
  as date) WHERE id = \$1;
---
kind: tools
name: cancel-hotel
type: postgres-sql
source: my-pg-source
description: Cancel a hotel by its ID.
parameters:
  - name: hotel_id
    type: string
    description: The ID of the hotel to cancel.
statement: UPDATE hotels SET booked = B'0' WHERE id = \$1;
---
kind: toolsets
name: my-toolset
tools:
  - search-hotels-by-name
  - search-hotels-by-location
  - book-hotel
  - update-hotel
  - cancel-hotel
"""

In [ ]:
# Write the file content into the tools file.
! echo "{file_content}" > "{tools_file_name}"

In [ ]:
TOOLS_FILE_PATH = f"/content/{tools_file_name}"

In [ ]:
# Start a toolbox server
! nohup {TOOLBOX_BINARY_PATH} --tools-file {TOOLS_FILE_PATH} -p {SERVER_PORT} > toolbox.log 2>&1 &

In [ ]:
# Check if toolbox is running
!sudo lsof -i :{SERVER_PORT}

## Step 3: Connect your agent to Toolbox

In this section, you will
1. Establish a connection to the tools by creating a Toolbox client.
2. Build an agent that leverages the tools and an LLM for Hotel Booking functionality.


> You need to authenticate as an IAM user so this notebook can access your Google Cloud Project. This access is necessary to use Google's LLM models.

In [ ]:
# Run this and allow access through the pop-up
from google.colab import auth

auth.authenticate_user()

In [ ]:
# @markdown Please fill in the value below with your GCP project ID and then run the cell.

# Please fill in these values.
project_id = ""  # @param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"

# Configure gcloud.
!gcloud config set project {project_id}

> You can either use LangGraph or LlamaIndex to develop a Toolbox based
> application. Run one of the sections below
> - [Connect using Google GenAI](#scrollTo=Fv2-uT4mvYtp)
> - [Connect using ADK](#scrollTo=QqRlWqvYNKSo)
> - [Connect Using LangGraph](#scrollTo=pbapNMhhL33S)
> - [Connect using LlamaIndex](#scrollTo=04iysrm_L_7v)


### Connect Using ADK

In [ ]:
! pip install google-adk[toolbox] --quiet

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService
from google.adk.tools.toolbox_toolset import ToolboxToolset
from google.genai import types

import os
# TODO(developer): replace this with your Google API key
os.environ['GOOGLE_API_KEY'] = "<GOOGLE_API_KEY>"

# Configure toolset
toolset = ToolboxToolset(
    server_url="http://127.0.0.1:5000",
    toolset_name="my-toolset"
)

prompt = """
  You're a helpful hotel assistant. You handle hotel searching, booking and
  cancellations. When the user searches for a hotel, mention it's name, id,
  location and price tier. Always mention hotel ids while performing any
  searches. This is very important for any operations. For any bookings or
  cancellations, please provide the appropriate confirmation. Be sure to
  update checkin or checkout dates if mentioned by the user.
  Don't ask for confirmations from the user.
"""

root_agent = Agent(
    model='gemini-2.0-flash-001',
    name='hotel_agent',
    description='A helpful AI assistant.',
    instruction=prompt,
    tools=[toolset],
)

session_service = InMemorySessionService()
artifacts_service = InMemoryArtifactService()
session = await session_service.create_session(
    state={}, app_name='hotel_agent', user_id='123'
)
runner = Runner(
    app_name='hotel_agent',
    agent=root_agent,
    artifact_service=artifacts_service,
    session_service=session_service,
)

queries = [
    "Find hotels in Basel with Basel in it's name.",
    "Can you book the Hilton Basel for me?",
    "Oh wait, this is too expensive. Please cancel it and book the Hyatt Regency instead.",
    "My check in dates would be from April 10, 2024 to April 19, 2024.",
]

for query in queries:
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(session_id=session.id,
                        user_id='123', new_message=content)

    responses = (
      part.text
      for event in events
      for part in event.content.parts
      if part.text is not None
    )

    for text in responses:
      print(text)

### Connect Using LangGraph

In [ ]:
# Install the Toolbox Langchain package
!pip install toolbox-langchain --quiet
!pip install langgraph --quiet

# Install the Langchain llm package
# TODO(developer): replace this with another model if needed
! pip install langchain-google-vertexai --quiet
# ! pip install langchain-google-genai
# ! pip install langchain-anthropic

Create a LangGraph Hotel Agent which can Search, Book and Cancel hotels.

In [ ]:
from langgraph.prebuilt import create_react_agent
# TODO(developer): replace this with another import if needed
from langchain_google_vertexai import ChatVertexAI
# from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_anthropic import ChatAnthropic
from langgraph.checkpoint.memory import MemorySaver

from toolbox_langchain import ToolboxClient

prompt = """
  You're a helpful hotel assistant. You handle hotel searching, booking and
  cancellations. When the user searches for a hotel, mention it's name, id,
  location and price tier. Always mention hotel id while performing any
  searches. This is very important for any operations. For any bookings or
  cancellations, please provide the appropriate confirmation. Be sure to
  update checkin or checkout dates if mentioned by the user.
  Don't ask for confirmations from the user.
"""

queries = [
    "Find hotels in Basel with Basel in it's name.",
    "Can you book the Hilton Basel for me?",
    "Oh wait, this is too expensive. Please cancel it and book the Hyatt Regency instead.",
    "My check in dates would be from April 10, 2024 to April 19, 2024.",
]

async def run_application():
  # Create an LLM to bind with the agent.
  # TODO(developer): replace this with another model if needed
  model = ChatVertexAI(model_name="gemini-2.0-flash-001", project=project_id)
  # model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
  # model = ChatAnthropic(model="claude-3-5-sonnet-20240620")

  # Load the tools from the Toolbox server
  client = ToolboxClient("http://127.0.0.1:5000")
  tools = await client.aload_toolset()

  # Create a Langraph agent
  agent = create_react_agent(model, tools, checkpointer=MemorySaver())
  config = {"configurable": {"thread_id": "thread-1"}}
  for query in queries:
      inputs = {"messages": [("user", prompt + query)]}
      response = agent.invoke(inputs, stream_mode="values", config=config)
      print(response["messages"][-1].content)

await run_application()

### Connect using LlamaIndex

In [ ]:
# Install the Toolbox LlamaIndex package
!pip install toolbox-llamaindex --quiet

# Install the llamaindex llm package
# TODO(developer): replace this with another model if needed
! pip install llama-index-llms-google-genai --quiet
# ! pip install llama-index-llms-anthropic

Create a LlamaIndex Hotel Agent which can Search, Book and Cancel hotels.

In [ ]:
import asyncio
import os

from llama_index.core.agent.workflow import AgentWorkflow

from llama_index.core.workflow import Context

# TODO(developer): replace this with another import if needed
from llama_index.llms.google_genai import GoogleGenAI
# from llama_index.llms.anthropic import Anthropic

from toolbox_llamaindex import ToolboxClient

prompt = """
  You're a helpful hotel assistant. You handle hotel searching, booking and
  cancellations. When the user searches for a hotel, mention it's name, id,
  location and price tier. Always mention hotel ids while performing any
  searches. This is very important for any operations. For any bookings or
  cancellations, please provide the appropriate confirmation. Be sure to
  update checkin or checkout dates if mentioned by the user.
  Don't ask for confirmations from the user.
"""

queries = [
    "Find hotels in Basel with Basel in it's name.",
    "Can you book the Hilton Basel for me?",
    "Oh wait, this is too expensive. Please cancel it and book the Hyatt Regency instead.",
    "My check in dates would be from April 10, 2024 to April 19, 2024.",
]

async def run_application():
    # Create an LLM to bind with the agent.
    # TODO(developer): replace this with another model if needed
    llm = GoogleGenAI(
        model="gemini-2.0-flash-001",
        vertexai_config={"project": project_id, "location": "us-central1"},
    )
    # llm = GoogleGenAI(
    #     api_key=os.getenv("GOOGLE_API_KEY"),
    #     model="gemini-2.0-flash-001",
    # )
    # llm = Anthropic(
    #   model="claude-3-7-sonnet-latest",
    #   api_key=os.getenv("ANTHROPIC_API_KEY")
    # )

    # Load the tools from the Toolbox server
    client = ToolboxClient("http://127.0.0.1:5000")
    tools = await client.aload_toolset()

    # Create a LlamaIndex agent
    agent = AgentWorkflow.from_tools_or_functions(
        tools,
        llm=llm,
        system_prompt=prompt,
    )

    # Run the agent
    ctx = Context(agent)
    for query in queries:
        response = await agent.run(user_msg=query, ctx=ctx)
        print(f"---- {query} ----")
        print(str(response))

await run_application()

### Connect Using Google GenAI

In [ ]:
# Install the Toolbox Core package
!pip install toolbox-core --quiet

# Install the Google GenAI package
!pip install google-genai --quiet

Create a Google GenAI Application which can Search, Book and Cancel hotels.

In [ ]:
import asyncio

from google import genai
from google.genai.types import (
    Content,
    FunctionDeclaration,
    GenerateContentConfig,
    Part,
    Tool,
)

from toolbox_core import ToolboxClient

prompt = """
  You're a helpful hotel assistant. You handle hotel searching, booking and
  cancellations. When the user searches for a hotel, mention it's name, id,
  location and price tier. Always mention hotel id while performing any
  searches. This is very important for any operations. For any bookings or
  cancellations, please provide the appropriate confirmation. Be sure to
  update checkin or checkout dates if mentioned by the user.
  Don't ask for confirmations from the user.
"""

queries = [
    "Find hotels in Basel with Basel in it's name.",
    "Please book the hotel Hilton Basel for me.",
    "This is too expensive. Please cancel it.",
    "Please book Hyatt Regency for me",
    "My check in dates for my booking would be from April 10, 2024 to April 19, 2024.",
]


async def run_application():
    toolbox_client = ToolboxClient("http://127.0.0.1:5000")

    # The toolbox_tools list contains Python callables (functions/methods) designed for LLM tool-use
    # integration. While this example uses Google's genai client, these callables can be adapted for
    # various function-calling or agent frameworks. For easier integration with supported frameworks
    # (https://github.com/googleapis/mcp-toolbox-python-sdk/tree/main/packages), use the
    # provided wrapper packages, which handle framework-specific boilerplate.
    toolbox_tools = await toolbox_client.load_toolset("my-toolset")
    genai_client = genai.Client(
        vertexai=True, project=project_id, location="us-central1"
    )

    genai_tools = [
        Tool(
            function_declarations=[
                FunctionDeclaration.from_callable_with_api_option(callable=tool)
            ]
        )
        for tool in toolbox_tools
    ]
    history = []
    for query in queries:
        user_prompt_content = Content(
            role="user",
            parts=[Part.from_text(text=query)],
        )
        history.append(user_prompt_content)

        response = genai_client.models.generate_content(
            model="gemini-2.0-flash-001",
            contents=history,
            config=GenerateContentConfig(
                system_instruction=prompt,
                tools=genai_tools,
            ),
        )
        history.append(response.candidates[0].content)
        function_response_parts = []
        for function_call in response.function_calls:
            fn_name = function_call.name
            # The tools are sorted alphabetically
            if fn_name == "search-hotels-by-name":
                function_result = await toolbox_tools[3](**function_call.args)
            elif fn_name == "search-hotels-by-location":
                function_result = await toolbox_tools[2](**function_call.args)
            elif fn_name == "book-hotel":
                function_result = await toolbox_tools[0](**function_call.args)
            elif fn_name == "update-hotel":
                function_result = await toolbox_tools[4](**function_call.args)
            elif fn_name == "cancel-hotel":
                function_result = await toolbox_tools[1](**function_call.args)
            else:
                raise ValueError("Function name not present.")
            function_response = {"result": function_result}
            function_response_part = Part.from_function_response(
                name=function_call.name,
                response=function_response,
            )
            function_response_parts.append(function_response_part)

        if function_response_parts:
            tool_response_content = Content(role="tool", parts=function_response_parts)
            history.append(tool_response_content)

        response2 = genai_client.models.generate_content(
            model="gemini-2.0-flash-001",
            contents=history,
            config=GenerateContentConfig(
                tools=genai_tools,
            ),
        )
        final_model_response_content = response2.candidates[0].content
        history.append(final_model_response_content)
        print(response2.text)


asyncio.run(run_application())

### Observe the output

You can see that the `Hyatt Regency Basel` has been booked for the correct dates.

In [ ]:
%%shell

export PGPASSWORD=my-password
psql -h 127.0.0.1 -U toolbox_user -d toolbox_db --no-password << EOF
SELECT * from hotels;
EOF

## Optional: Cleanup

Executing this will terminate the processes running on the database and Toolbox ports.

This is necessary before re-running the startup cells for these services to prevent `port already in use` errors.

In [ ]:
!lsof -t -i :5432 | xargs kill -9

In [ ]:
# Verify that the database process is killed
!sudo lsof -i :5432